# Printing point clouds

In [ ]:
import subprocess
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.display import Video
import numpy as np
from tqdm.auto import tqdm
import astro3d
from astropy.visualization import ImageNormalize, LogStretch

## Load Data

In [ ]:
data = np.loadtxt('shocks.csv', delimiter=',')
xi, yi, zi, sigma, mach, = data[:, :5].T
del data

## Prepare data

Rescale to a maximum side-length of 5 cm

In [ ]:
Lreal = 5.0
debug = True

In [ ]:
Lx = xi.max() - xi.min()
Ly = yi.max() - yi.min()
Lz = zi.max() - zi.min()

Lmax = max(Lx, Ly, Lz)

Lx *= Lreal / Lmax
Ly *= Lreal / Lmax
Lz *= Lreal / Lmax

xi = (xi - xi.min()) / Lmax * Lreal
yi = (yi - yi.min()) / Lmax * Lreal
zi = (zi - zi.min()) / Lmax * Lreal

if debug:
    # for development: smaller z-range
    mask = (zi > Lreal/2 - 0.25) & (zi < Lreal/2 + 0.25)
    xi = xi[mask]
    yi = yi[mask]
    zi = zi[mask]
    mach = mach[mask]
    sigma = sigma[mask]
    
    zi = zi - zi.min()
    Lz = zi.max()

In [ ]:
stack = astro3d.image_stack.IStack([Lx, Ly, Lz])

In [ ]:
stack.nx, stack.ny, stack.nz

In [ ]:
xg = np.linspace(xi.min(), xi.max(), stack.nx)
yg = np.linspace(yi.min(), yi.max(), stack.ny)
zg = np.linspace(zi.min(), zi.max(), stack.nz)
del stack

In [ ]:
n_sigma = 5 # beyond how many sigma as distance we don't include this star in the slice

### Compute images with color information as weights

Here, every particle deposits some density weighted by the respective color information, e.g. an entirely green particle will deposity no density in the blue channel and so on. We use `RGB_cmap` which makes an existing colormap transition to white.

In [ ]:
cmap = astro3d.cmaps.RGB_cmap('magma')

# we shift by `shift` to have only the upper part of the color scale, more saturation and less white points
shift = 0.25
x_color = shift  + (1 - shift) * (mach - mach.min()) / (mach.max() - mach.min())
weights = cmap(x_color)

In [ ]:
idx = mach.argsort()

f, ax = plt.subplots()
pcm = ax.pcolormesh(mach[idx], [0, 1], [mach[idx], mach[idx]], fc='none')
pcm.set_facecolor(weights[idx, :])
pcm.set_array(None)
ax.set_aspect(0.1)
ax.set_xlabel('mach number')
ax.set_title('color mapping')
ax.set_yticks([]);

This runs about 12 minutes on 8 cores (M1) for a 5cm full res image stack

In [ ]:
%%time
astro3d.fmodule.numthreads = 8
image, alpha = astro3d.image_stack.image_stack_from_point_cloud(
    xi, yi, zi, xg=xg, yg=yg, zg=zg, sigmas=sigma, weights=weights, alpha_method=True)

In [ ]:
%%time
astro3d.fmodule.numthreads = 8
image = astro3d.image_stack.image_stack_from_point_cloud(xi, yi, zi, xg=xg, yg=yg, zg=zg, sigmas=sigma, weights=weights)
image = image.astype(np.float32)

In [ ]:
%%time
image = np.load('image.npz')['image']

## Plotting

### Normalize

Normalize the image (keep a copy of the original in `image_o`).  
**Runtime of 90 minutes!**

A first approach.

```python
%%time
maxval = 100 * image.mean()
image_norm = image / maxval
image_norm[image_norm>1.0] = 1.0
````

Second approach.  
**Note:** This takes around 21 min.

In [ ]:
%%time
maxval = image.max() / 10
print(f'maxval = {maxval}')
image = np.array(ImageNormalize(vmin=0, vmax=maxval, clip=True, stretch=LogStretch(a=1))(image))

## Save the dataset

we convert this to 0...255 unsigned integer to save on file size (factor of 100!)

In [ ]:
data = (255 * image).astype(np.uint8)

print(f'image is {image.nbytes / 1024**3:.2g} GB')
print(f'data is {data.nbytes / 1024**3:.2g} GB')

del image

In [ ]:
np.savez_compressed('data.npz', rho=data, x=xg, y=yg, z=zg)

### Single plot

In [ ]:
iz = data.shape[2] // 2
vmax = data.max()

f, ax = plt.subplots(figsize = (6,6))
if data.ndim == 3:
    cc = ax.pcolormesh(xg, yg, data[:, :, iz].T, vmax=vmax, vmin=0)
elif data.ndim == 4:
    cc = ax.imshow(data[:, :, iz, :].transpose(1, 0, 2), extent=[xg[0], xg[-1], yg[0], yg[-1]], origin='lower')

# scatter points
#alpha_max = 0.01   # maximum alpha of the scatter points. Set to 0 to turn off
#mask = np.abs(zi - zg[iz]) < n_sigma * sigma
#alphas = np.exp(-((zi[mask] - zg[iz])/(np.sqrt(2) * sigma[mask]))**2) * alpha_max
#sc = ax.scatter(xi[mask], yi[mask], c='r', s=2, alpha=alphas)

ti = ax.text(0.03, 0.96, f'iz = {iz}', c='w',  transform=ax.transAxes, ha='left', va='top')
ax.set_aspect(1)
ax.set_facecolor('g')

Define update function for making movie

In [ ]:
def update(iz):
    # update density
    if image.ndim == 3:
        cc.set_array(data[:, :, iz].T.ravel())
    elif image.ndim == 4:
        cc.set_data(data[:, :, iz, :].transpose(1, 0, 2))
    # update scatter
    if alpha_max > 0.0:
        mask = np.abs(zi - zg[iz]) < n_sigma * sigma
        if mask.sum()==0:
            sc.set_alpha(np.zeros(len(sc.get_offsets())))
        else:
            alphas = np.exp(-((zi[mask] - zg[iz])/(np.sqrt(2) * sigma[mask]))**2) * alpha_max
            sc.set_offsets(np.c_[xi[mask], yi[mask]])
            sc.set_alpha(alphas)
    
    ti.set_text(f'iz = {iz}')

## Make movie parallel

This is a very simple paralellization that doesn't work with `multiprocessing`, but does work with `multiprocess`. In my test, this brought down the movie generation from 2m8s to 0m22s (included about 6 seconds for images->movie).

In [ ]:
parallel = True # whether to compile the movie in parallel or not

In [ ]:
%%time 
fpath = Path('frames')
fpath.mkdir(exist_ok=True)

if parallel:
    import multiprocess as mp
    
    # the worker function that the workers execute
    def work(iz):
        update(iz)
        f.savefig(fpath / f'frame_{iz:03d}.png', transparent=False, dpi=300)
        
    # create a pool
    p = mp.Pool(processes=mp.cpu_count())
    res = p.map(work, range(len(zg)))
else:
    # normal serial loop
    for iz in tqdm(range(len(zg))):
        update(iz)
        f.savefig(fpath / f'frame_{iz:03d}.png', transparent=False, dpi=300)

ret = subprocess.check_output(
    (f'ffmpeg -y -framerate 15 -i {fpath}/frame_%03d.png -c:v libx264 -crf 23 -pix_fmt yuv420p output_mp.mp4').split(), stderr=subprocess.STDOUT)

In [ ]:
Video('output_mp.mp4', width=500, html_attributes='autoplay controls')

# Dithering to image slices

Set output path

In [ ]:
path = Path(astro3d.get_output()) / 'shock_surfaces_v2'
path.mkdir(exist_ok=True)

Set a filling fraction of the dithering

In [ ]:
fill = 0.3

define a color palette matching CMYK but with white as black

In [ ]:
palette = np.array(astro3d.image_stack.vero_palette)[[2,3,4,1,0]]
ax = plt.imshow([palette]).axes
ax.axis('off')
ax.figure.set_facecolor('0.75')

Display a single slice

In [ ]:
iz = data.shape[2] // 2
slice = data[:, :, iz, :]

In [ ]:
# dither image
imd = astro3d.image_stack.dither_palette(slice, palette)

In [ ]:
# brighten and apply alpha
imdb = astro3d.image_stack.dither_brighten(imd, 1-fill, bg=3 * [128])

In [ ]:
def showslice(slice, ax=None, title=None):
    if ax is None:
        f, ax = plt.subplots()
    ax.imshow(slice.transpose(1, 0, 2), origin='lower')
    ax.set_aspect(2)
    ax.set_facecolor('g')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(9, 3), dpi=150, gridspec_kw={'wspace':0.05})
showslice(slice, axs[0], 'original')
showslice(imd,   axs[1], 'dithered RGBA')
showslice(imdb,  axs[2], 'dithered RGB')

In [ ]:
import imageio

In [ ]:
def worker_function(ranges):
    # loop over the given ranges
    iz0, iz1 = ranges
    for iz in range(iz0, iz1):

        # get the slice
        slice = data[:, :, iz, :]
    
        # dither image
        imd = astro3d.image_stack.dither_palette(slice, palette)
    
        # dither alpha
        imdb = astro3d.image_stack.dither_brighten(imd, 1 - fill, bg=3*[128])

        # save as png
        if path is not None:
            imageio.imwrite(path / f'slice_{iz:04d}.png', imdb.transpose(1, 0, 2)[::-1, :, :])

    return 0

### Serial execution

```python
%%time
worker_function([0, data.shape[2]])
```

### Parallel execution

To be able to do this inside a notebook, we use `multiprocess` since `multiprocessing` will not be able to pickle the worker function

In [ ]:
import multiprocess as mp

In [ ]:
# define the number of workers, and the ranges on which each is working
n_workers = mp.cpu_count()
chunk_size = int(data.shape[2] / n_workers)
ranges = [[i * chunk_size, (i + 1) * chunk_size] for i in range(n_workers)]
ranges[-1][-1] = data.shape[2]

Execute

In [ ]:
%%time
with mp.get_context().Pool() as pool:
    pool.map(worker_function, ranges)

# Analyze Stack

In [ ]:
stack = astro3d.image_stack.IStack(path)
stack.show_colors(titles=['VeroWhite', 'VeroYellow', 'UltraClear', 'VeroCyan', 'VeroMagenta', 'VeroBlack'])

In [ ]:
stack.empty_indices = [2]

In [ ]:
f, axs = stack.three_views(bg=3 * [200])
f.dpi = 400

Try improving things

In [ ]:
stack.replace_color(5, 3 * [255])

In [ ]:
stack.show_colors(titles=['VeroWhite', 'VeroYellow', 'UltraClear', 'VeroCyan', 'VeroMagenta'])

In [ ]:
stack.empty_indices = [2]

In [ ]:
f, axs = stack.three_views(bg=3 * [200])
f.dpi = 400